In [150]:
import pandas as pd
from pandas import DataFrame
import dateutil.parser
import json


view_party=pd.read_csv('../refdata/view_party.csv').drop(['cmp','euprofiler','ees','castles_mair','huber_inglehart','ray','benoit_laver','chess'],axis=1)
partyColumns=view_party.columns

view_election=pd.read_csv('../refdata/view_election.csv')
electionColumns=view_election.columns

view_election=view_election.merge(view_party,on='party_id',how='left',suffixes=('','_2'))
view_election=view_election.drop([x for x in view_election.columns if x.endswith('_2')],axis=1)
view_election.election_date=view_election.election_date.apply(dateutil.parser.parse)

view_cabinet=pd.read_csv('../refdata/view_cabinet.csv')
cabinetColumns=view_cabinet.columns

view_cabinet=view_cabinet.merge(view_party,on='party_id',how='left',suffixes=('','_2'))
view_cabinet=view_cabinet.drop([x for x in view_cabinet.columns if x.endswith('_2')],axis=1)
view_cabinet=view_cabinet.merge(view_election,on=['election_id','party_id'],how='left',suffixes=('','_2'))
view_cabinet=view_cabinet.drop([x for x in view_cabinet.columns if x.endswith('_2')],axis=1)
view_cabinet.election_date=view_cabinet.election_date.apply(dateutil.parser.parse)


NationalElections=view_election[view_election.election_type=='parliament']
EuroElections=view_election[view_election.election_type=='ep']

def getLastElection(t):
    return t[t.election_date==t.election_date.max()]

def getLastCabinet(t):
    return t[t.start_date==t.start_date.max()]

NationalElections=NationalElections.groupby('country_name_short').apply(getLastElection).reset_index(drop=True)
EuroElections=EuroElections.groupby('country_name_short').apply(getLastElection).reset_index(drop=True)
Cabinets=view_cabinet.groupby('country_name_short').apply(getLastCabinet).reset_index(drop=True)

NationalElections=NationalElections[~NationalElections.country_name_short.isin(['AUS','CAN','ISR','NZL','JPN'])]
EuroElections=EuroElections[~EuroElections.country_name_short.isin(['AUS','CAN','ISR','NZL','JPN'])]
Cabinets=Cabinets[~Cabinets.country_name_short.isin(['AUS','CAN','ISR','NZL','JPN'])].drop(['election_type','previous_cabinet_id','previous_parliament_election_id'],axis=1)

NationalElections.to_csv('../refdata/national_elections.csv',index=False)
EuroElections.to_csv('../refdata/euro_elections.csv',index=False)
Cabinets.to_csv('../refdata/cabinets.csv',index=False)

CPI_2015_data=pd.read_excel('../build/CPI_2015_data.xlsx')
CPI_2015_data=CPI_2015_data[CPI_2015_data.Region.isin(['ECA','WE/EU'])]

WDI_data=pd.read_csv('../build/WDI_data.csv')
WDI_data=WDI_data[['Country Code','Indicator Code','2015']][WDI_data['Country Code'].isin(CPI_2015_data.wbcode)] \
    .set_index(['Country Code','Indicator Code']).unstack()['2015'][['SP.POP.TOTL','NY.GDP.MKTP.CD', 'NY.GDP.PCAP.CD']] \
    .reset_index().rename(columns={
            'Country Code':'countryCode',
            'SP.POP.TOTL':'totalPopulation',
            'NY.GDP.MKTP.CD':'gdpUsd',
            'NY.GDP.PCAP.CD':'gdpPerCapitaUsd',
        }).rename_axis(None,axis=1)
    
Country=pd.read_csv('../refdata/country.csv')[['name_short','oecd_accession_date','eu_accession_date']]

def getPartyJson(code):
    r = {}
        
    if code in Cabinets.country_name_short.tolist():
        topLevelFields=['country_id','country_name_short','country_name','election_id','election_date','seats_total',
                   'cabinet_id','cabinet_name','start_date','caretaker','election_seats_total']
        nat = Cabinets[code==Cabinets.country_name_short] \
            .sort_values(['seats','vote_share','party_name_short'],ascending=False)
        nat.seats_total=nat.seats_total.fillna(-1)
        t=nat[topLevelFields].reset_index(drop=True).ix[0].to_dict()
        t['election_date']=str(t['election_date'])
        r.update(t)
        r.update({'parties' : [x[1][nat.columns.difference(topLevelFields)].dropna().to_dict() for x in nat.iterrows()]})    
    if code in CPI_2015_data.wbcode.tolist():
        r['cpi2015'] = CPI_2015_data[CPI_2015_data.wbcode==code].CPI2015.tolist()[0]
    if code in WDI_data.countryCode.tolist():
        r.update(WDI_data[WDI_data.countryCode==code].reset_index(drop=True).T.drop('countryCode')[0].dropna().to_dict())
    if code in Country.name_short.tolist():        
        r.update(Country[Country.name_short==code].reset_index(drop=True).T.drop('name_short')[0].dropna().to_dict())
        
    return r

data={code:getPartyJson(code) for code in  CPI_2015_data.wbcode.drop_duplicates().tolist()+['ROU'] if not code in ['AUS','CAN','ISR','NZL','JPN']}

with open('../public/parlgov.json', 'w') as outfile:
    json.dump(data, outfile,indent=4, default='')

In [148]:
 CPI_2015_data.wbcode.drop_duplicates().tolist()+['ROM']

[u'DNK',
 u'FIN',
 u'SWE',
 u'NLD',
 u'NOR',
 u'CHE',
 u'DEU',
 u'LUX',
 u'GBR',
 u'ISL',
 u'BEL',
 u'AUT',
 u'IRL',
 u'EST',
 u'FRA',
 u'PRT',
 u'POL',
 u'CYP',
 u'LTU',
 u'SVN',
 u'ESP',
 u'CZE',
 u'MLT',
 u'LVA',
 u'GEO',
 u'HRV',
 u'HUN',
 u'SVK',
 u'GRC',
 u'ROM',
 u'ITA',
 u'MON',
 u'MKD',
 u'TUR',
 u'BGR',
 u'SCG',
 u'BIH',
 u'ALB',
 u'ARM',
 u'LWI',
 u'MDA',
 u'BLR',
 u'AZE',
 u'RUS',
 u'KAZ',
 u'KGZ',
 u'UKR',
 u'TJK',
 u'UZB',
 u'TKM',
 'ROM']

In [88]:
print 'CHE'in Country.name_short.tolist()

Country[Country.name_short=='CHE'].reset_index(drop=True).T.drop('name_short')[0].dropna().to_dict()

True


{'oecd_accession_date': '1961-09-28'}

In [29]:
CPI_2015_data=pd.read_excel('../build/CPI_2015_data.xlsx')
CPI_2015_data=CPI_2015_data[CPI_2015_data.Region.isin(['ECA','WE/EU'])]

WDI_data=pd.read_csv('../build/WDI_data.csv')
WDI_data=WDI_data[['Country Code','Indicator Code','2015']][WDI_data['Country Code'].isin(CPI_2015_data.wbcode)] \
    .set_index(['Country Code','Indicator Code']).unstack()['2015'][['SP.POP.TOTL','NY.GDP.MKTP.CD', 'NY.GDP.PCAP.CD']] \
    .reset_index().rename(columns={
            'Country Code':'countryCode',
            'SP.POP.TOTL':'totalPopulation',
            'NY.GDP.MKTP.CD':'gdpUsd',
            'NY.GDP.PCAP.CD':'gdpPerCapitaUsd',
        }).rename_axis(None,axis=1)

In [100]:
CPI_2015_data

,Rank,CPI2015,Country,Region,wbcode,World Bank CPIA,World Economic Forum EOS,Bertelsmann Foundation TI,African Dev Bank,IMD World Competitiveness Yearbook,...,CPI2015(2),Rank2,Number of Sources,Std Deviation of Sources,Standard Error,Minimum,Maximum,Lower CI,Upper CI,Country2
0,1,91,Denmark,WE/EU,DNK,NaN,89.0,NaN,NaN,96.0,...,91,1,7,5.71,2.16,83,98,87.4360,94.5640,Denmark
1,2,90,Finland,WE/EU,FIN,NaN,93.0,NaN,NaN,91.0,...,90,2,7,4.69,1.77,83,98,87.0795,92.9205,Finland
2,3,89,Sweden,WE/EU,SWE,NaN,89.0,NaN,NaN,87.0,...,89,3,7,4.53,1.71,83,98,86.1785,91.8215,Sweden
4,5,87,Netherlands,WE/EU,NLD,NaN,84.0,NaN,NaN,85.0,...,87,5,7,4.79,1.81,83,97,84.0135,89.9865,Netherlands
5,5,87,Norway,WE/EU,NOR,NaN,92.0,NaN,NaN,84.0,...,87,5,7,7.93,3.00,73,98,82.0500,91.9500,Norway
6,7,86,Switzerland,WE/EU,CHE,NaN,87.0,NaN,NaN,88.0,...,86,7,6,6.25,2.55,73,89,81.7925,90.2075,Switzerland
9,10,81,Germany,WE/EU,DEU,NaN,72.0,NaN,NaN,83.0,...,81,10,7,6.62,2.50,72,89,76.8750,85.1250,Germany
10,10,81,Luxembourg,WE/EU,LUX,NaN,89.0,NaN,NaN,85.0,...,81,10,5,13.45,6.02,57,89,71.0670,90.9330,Luxembourg
11,10,81,United Kingdom,WE/EU,GBR,NaN,82.0,NaN,NaN,81.0,...,81,10,7,6.36,2.40,73,89,77.0400,84.9600,United Kingdom
13,13,79,Iceland,WE/EU,ISL,NaN,87.0,NaN,NaN,83.0,...,79,13,5,10.14,4.53,65,89,71.5255,86.4745,Iceland


In [68]:
code='AUT'

if code in CPI_2015_data.wbcode:
    print 1

WDI_data[WDI_data.countryCode==code].reset_index(drop=True).T.drop('countryCode')[0].to_dict()

{'gdpPerCapitaUsd': 43438.8630381343,
 'gdpUsd': 374055872241.322,
 'totalPopulation': 8611088.0}

In [79]:
%%bash
ls ../refdata/*.csv

../refdata/cabinets.csv
../refdata/country.csv
../refdata/euro_elections.csv
../refdata/europe_country_doc.csv
../refdata/europe_election_doc.csv
../refdata/europe_party_doc.csv
../refdata/external_country_iso.csv
../refdata/national_elections.csv
../refdata/politician_president.csv
../refdata/view_cabinet.csv
../refdata/view_election.csv
../refdata/view_party.csv


In [146]:
PoliticianPresident=pd.read_csv('../refdata/politician_president.csv')
#ExternalCountry=ExternalCountry[(ExternalCountry.continent=='Europe')|(ExternalCountry.iso3.isin(['RUS','CYP','CYN','TUR']))]
PoliticianPresident.start_date=PoliticianPresident.start_date.apply(dateutil.parser.parse)

PoliticianPresident=PoliticianPresident[PoliticianPresident.end_date.isnull()].sort_values('start_date')
PoliticianPresident
Cabinets.merge(PoliticianPresident,on='country_id',how='left',suffixes=('_','')).merge(Cabinets,on='party_id',how='left',suffixes=('_',''))[['country_name','person_id_source','party_id','party_name']].drop_duplicates()

,country_name,person_id_source,party_id,party_name
0,Austria,NaN,1429.0,Die Grünen – Die Grüne Alternative
6,NaN,NaN,NaN,NaN
19,Bulgaria,Rosen Plevneliev,1541.0,Grazhdani za Evropeysko Razvitie na Balgariya
27,Switzerland,Didier Burkhalter,26.0,Freisinnig-Demokratische Partei der Schweiz – ...
39,Cyprus,Nicos Anastasiades,851.0,Dimokratikó Kómma
45,NaN,Miloš Zeman,406.0,NaN
53,NaN,Joachim Gauck,610.0,NaN
84,Estonia,Toomas Hendrik Ilves,1448.0,Sotsiaaldemokraatlik Erakond | Mõõdukad
90,Finland,Sauli Niinistö,1118.0,Kansallinen Kokoomus – Samlingspartiet
99,France,François Hollande,1539.0,Parti socialiste
